In [1]:
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

./combined_data.csv
./exercise.csv
./calories.csv
./.config/default_configs.db
./.config/active_config
./.config/.last_opt_in_prompt.yaml
./.config/config_sentinel
./.config/.last_update_check.json
./.config/.last_survey_prompt.yaml
./.config/gce
./.config/logs/2023.10.24/13.48.54.315038.log
./.config/logs/2023.10.24/13.49.12.436305.log
./.config/logs/2023.10.24/13.48.27.345883.log
./.config/logs/2023.10.24/13.49.03.705354.log
./.config/logs/2023.10.24/13.49.22.070747.log
./.config/logs/2023.10.24/13.49.22.844424.log
./.config/configurations/config_default
./sample_data/README.md
./sample_data/anscombe.json
./sample_data/mnist_train_small.csv
./sample_data/mnist_test.csv
./sample_data/california_housing_test.csv
./sample_data/california_housing_train.csv


In [2]:
# Import Calories Dataset
df_cal = pd.read_csv('calories.csv')

# Import Exercises Dataset
df_ex  = pd.read_csv('exercise.csv')

# Merging Datasets
df = pd.merge(df_ex, df_cal, on = 'User_ID')

# Get dummies
df['Gender'] = pd.get_dummies(df['Gender'], prefix_sep='_', drop_first=True)

def bmi_cal(weight, height):
    w = weight
    h = height/100
    bmi = w/(h*h)
    return round(bmi,2)
df['bmi'] = df.apply(lambda x: bmi_cal(x['Weight'],x['Height']),axis=1)

df.head()


,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,bmi
0,14733363,1,68,190.0,94.0,29.0,105.0,40.8,231.0,26.04
1,14861698,0,20,166.0,60.0,14.0,94.0,40.3,66.0,21.77
2,11179863,1,69,179.0,79.0,5.0,88.0,38.7,26.0,24.66
3,16180408,0,34,179.0,71.0,13.0,100.0,40.5,71.0,22.16
4,17771927,0,27,154.0,58.0,10.0,81.0,39.8,35.0,24.46


Dense Neural Network


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Load dataset
df = df

# Preprocess data
X = df[['Duration', 'Heart_Rate', 'Body_Temp']]
y = df['Calories']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, 'DNN_scaler.save')

# Build model
model = models.Sequential([
    layers.Input(shape=(3,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))

# Save model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('DNN_model.tflite', 'wb') as f:
    f.write(tflite_model)


Epoch 1/100
375/375 [==============================] - 2s 3ms/step - loss: 3898.0002 - mae: 40.9856 - val_loss: 277.8774 - val_mae: 12.8167
Epoch 2/100
375/375 [==============================] - 1s 2ms/step - loss: 225.0989 - mae: 11.0847 - val_loss: 190.4801 - val_mae: 9.9014
Epoch 3/100
375/375 [==============================] - 1s 2ms/step - loss: 171.0504 - mae: 9.2590 - val_loss: 166.9002 - val_mae: 9.1992
Epoch 4/100
375/375 [==============================] - 1s 2ms/step - loss: 155.3539 - mae: 8.6154 - val_loss: 157.3720 - val_mae: 8.6612
Epoch 5/100
375/375 [==============================] - 1s 2ms/step - loss: 152.0795 - mae: 8.4478 - val_loss: 156.0201 - val_mae: 8.5563
Epoch 6/100
375/375 [==============================] - 1s 2ms/step - loss: 150.8600 - mae: 8.3630 - val_loss: 155.6350 - val_mae: 8.4605
Epoch 7/100
375/375 [==============================] - 1s 2ms/step - loss: 150.6553 - mae: 8.3478 - val_loss: 155.8431 - val_mae: 8.5513
Epoch 8/100
375/375 [================

In [6]:
import numpy as np
import joblib

# Load the scaler
scaler = joblib.load('DNN_scaler.save')

# Prepare input data
duration = 120  # example duration in minutes
heart_rate = 110  # example heart rate in bpm
body_temp = 38  # example body temperature in Fahrenheit

input_data = np.array([[duration, heart_rate, body_temp]], dtype=np.float32)
input_data_scaled = scaler.transform(input_data)

import tflite_runtime.interpreter as tflite

# Load the TFLite model
interpreter = tflite.Interpreter(model_path='DNN_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
interpreter.set_tensor(input_details[0]['index'], input_data_scaled)

# Invoke the interpreter
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)


[[784.9695]]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
# Import Exercises Dataset
df_sports  = pd.read_csv('combined_data.csv')
df_exercise = df_sports[['Activity',	'Set #',	'Weight',	'Reps']]

unique_activities = df_exercise['Activity'].unique()
mets_dict = {
    'Lat Pull Down': 3.0,
    'Shoulder Press': 3.0,
    'Incline Bench Press': 3.0,
    'Leg Press': 3.0,
    'Bicep Curl': 3.0,
    'Squat': 3.5,
    'Deadlift': 3.5,
    'Squats': 3.5,
    'Ab Crunch': 3.0,
    'Barbell Overhead Press': 3.0,
    'Barbell Triceps': 3.0,
    'Diamond Press Up': 3.0,
    'Bench Press': 3.0,
    'Barbell Row': 3.0,
    'Treadmill (light)': 2.0,
    'Treadmill (moderate)': 6.0,
    'Treadmill (vigorous)': 9.0,
    'Chest Press': 3.0,
    'Bilateral Arm Curl': 3.0,
    'Triceps Extension': 3.0,
    'Ab Work': 3.0,
    'Pull Ups': 8.0,
    'Tricep Curl': 3.0,
    'Bike (light)': 4.0,
    'Bike (moderate)': 6.0,
    'Bike (vigorous)': 8.0,
    'Elliptical': 5.0,
    'Leg Curl': 3.0,
    'Lifted Crunches': 3.0,
    'Angled Situps': 3.0,
    'Deadlifts': 3.5,
    'Sqat (Restricted Machine)': 3.5,
    'Assisted Pullups': 3.0,
    'Assisted Dips': 3.0,
    'Pec Fly': 3.0,
    'Barbell Rows': 3.0,
    'Ab Crunches': 3.0,
    'Seated Row': 3.0,
    'Deltoid Raise': 3.0,
    'Tricep Extension': 3.0,
    'Push Ups': 8.0,
    'Arm Curl': 3.0,
    'Arm Extension': 3.0,
    'Free Butterfly': 3.0,
    'Bicep Curls': 3.0,
    'Tricep Curls': 3.0,
    'Dead Lift': 3.5,
    'Ad Work': 3.0,
    'Front Squats': 3.5,
    'Dumbell Curls': 3.0,
    'Skipping': 12.0,
    'Back Extension': 3.0,
    'Leg Extension': 3.0,
    'Pull Up': 8.0,
    'Triceps': 3.0,
    'Crunch': 3.0,
    'Kood Kick': 3.0,
    'Row': 3.0,
    'Crunches': 3.0,
    'Ab Roll': 3.0,
    'Bench': 3.0,
    'Shoulder': 3.0,
    'Arc': 3.0,
    'Goat Balls': 3.0,  # Assumed general moderate effort
    'Shoulder Shrugs': 3.0,
    'Tricep Dips': 3.0,
    'Pullups Bicep': 3.0,
    'Pullups Lats': 3.0,
}

df_exercise['METs'] = df_exercise['Activity'].map(mets_dict)

df_exercise.head()

<ipython-input-23-4ad1580156e1>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exercise['METs'] = df_exercise['Activity'].map(mets_dict)


,Activity,Set #,Weight,Reps,METs
0,Lat Pull Down,1.0,35.0,11.0,3.0
1,Lat Pull Down,2.0,42.5,10.0,3.0
2,Lat Pull Down,3.0,50.0,7.0,3.0
3,Shoulder Press,1.0,30.0,10.0,3.0
4,Shoulder Press,2.0,40.0,9.0,3.0


In [52]:
import pandas as pd
import math
def distribute_workout(predicted_calories, user_weight):
    # MET values for different exercises
    mets_dict = {
        'deadlift': 3.5,
        'squat': 3.5,
        'shoulder_press': 3.0,
        'cardio_running': 7.0,
    }

    # Initial distribution of workout (4 sets for strength exercises, 20 minutes for cardio)
    workout_plan = {
        'deadlift': {'sets': 4, 'reps': 8, 'weight': 60},  # 60kg for deadlift
        'squat': {'sets': 4, 'reps': 10, 'weight': 50},   # 50kg for squat
        'shoulder_press': {'sets': 4, 'reps': 12, 'weight': 20},  # 20kg for shoulder press
        'cardio_running': {'duration': 20},  # 20 minutes of running
    }

    # Calculate the estimated calories burned for each exercise
    total_calories = 0
    for exercise, details in workout_plan.items():
        if 'sets' in details and 'reps' in details:
            # Strength exercises
            total_reps = details['sets'] * details['reps']
            met_value = mets_dict[exercise]
            # Estimating calories burned for strength exercises is complex and depends on various factors.
            # This is a very rough approximation for illustration purposes.
            calories = (met_value * (3.5 * user_weight) / 200) * (total_reps / 10)
        elif 'duration' in details:
            # Cardio exercises
            met_value = mets_dict[exercise]
            calories = met_value * (3.5 * user_weight / 200) * details['duration']
        else:
            calories = 0
        total_calories += calories

    # Normalize the workout plan to meet the predicted calorie goal
    calorie_ratio = math.log(predicted_calories/total_calories)

    for exercise, details in workout_plan.items():
        if 'sets' in details and 'reps' in details:
            # Adjust weight for strength exercises
            details['weight'] = round(details['weight'] * calorie_ratio)
        elif 'duration' in details:
            # Adjust duration for cardio exercises
            details['duration'] = round(details['duration'] * calorie_ratio)

    return workout_plan

# Example usage
user_weight = 90
predicted_calories = 2000
workout_plan = distribute_workout(predicted_calories,user_weight)
print(workout_plan)


{'deadlift': {'sets': 4, 'reps': 8, 'weight': 117}, 'squat': {'sets': 4, 'reps': 10, 'weight': 98}, 'shoulder_press': {'sets': 4, 'reps': 12, 'weight': 39}, 'cardio_running': {'duration': 39}}
